In [1]:
# Import the parent directory into the path
import sys
sys.path.append("..")

In [5]:
# from src.data.data_loader import get_train_dataset
from torchvision import datasets, transforms
import torch.nn as nn
import pprint
import torch
import torch.optim as optim
import torch.nn.functional as F
from src.models.model import ShapeClassifier

from src.configs.model_config import ModelConfig
# from src.data.data_loader import train_loader, num_classes
# from src.utils.logs import writer
from src.utils.train import train
from src.utils.test import test
import wandb
import json

In [ ]:
wandb.login()

In [ ]:
sweep_config = {
    'method': 'bayes',
    "metric": {
        "name": 'accuracy',
        "goal": 'maximize'
    },
    "parameters": {
        'learning_rate': {
            # a flat distribution between 0 and 0.1
            'distribution': 'uniform',
            'min': 0,
            'max': 0.001
        },
        'batch_size': {
            # integers between 32 and 256
            # with evenly-distributed logarithms
            'distribution': 'q_log_uniform_values',
            'q': 8,
            'min': 32,
            'max': 256,
        },
        'epochs': {
            'values': [15, 20, 30]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
        'fc_layer_size': {
            'values': [128, 256, 512]
        },
    }
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from src.data.data_loader import get_train_dataset


def build_dataset(batch_size):

    return get_train_dataset(batch_size)


def build_network(fc_layer_size):
    network = ShapeClassifier(num_classes=3,hidden_size=fc_layer_size)

    return network.to(device)


def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = F.cross_entropy(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader)

def validate(network, loader):
    network.eval()
    correct = 0
    with torch.no_grad():
        for _, (data, target) in enumerate(loader):
            data, target = data.to(device), target.to(device)
            output = network(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    return correct / len(loader.dataset)
    
def train(config=None):
    # Initialize a new wandb run  

    with wandb.init():
        config = wandb.config
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        print("config" , config)
   
        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size)
        optimizer = build_optimizer(
            network, config.optimizer, config.learning_rate)


        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            acc = validate(network, loader)
            print(f"Epoch {epoch} avg loss: {avg_loss} accuracy: {acc}")
            wandb.log({"loss": avg_loss, "epoch": epoch, "accuracy": acc})

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

In [ ]:
wandb.agent(sweep_id, train, count=20)